# [Boosters] Raiffeisen Data Cup. Baseline
Общий подход:
- Добавляем к каждой транзакции столбец: is_work (если транзакция находится в пределах 0.02 от дома клиента)
- Добавляем к каждой транзакции столбец: is_home (если транзакция находится в пределах 0.02 от работы клиента)
- Обучаем классификатор предсказывающий вероятность (is_home == 1) для транзакции
- Обучаем классификатор предсказывающий вероятность (is_work == 1) для транзакции

Точность определения местоположения:
- для классификатора is_home: ~3x%
- для классификатора is_work: ~2x%
- общая оценка на Public Leaderboard: ???

Примечание
* Требуется Python версии 3.5
* Требуется библиотека xgboost (для обучения использовалась xgboost версии 0.7.post3)
* Требуются файлы: test_set.csv, train_set.csv в одном каталоге с данным скриптом
* Требования к памяти: должно работать с 2Гб свободного RAM
* Время работы: ~3 минуты (тестировалось на процессоре Intel Core i7-4770)

In [1]:
%load_ext autoreload
%autoreload 2

import sys
MODULES_PATH = '../code/'
if MODULES_PATH not in sys.path:
    sys.path.append(MODULES_PATH)
import mfuncs
    
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
pd.options.display.max_columns = 1000

import lightgbm as lgb


from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth, AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Определим типы колонок для экономии памяти
dtypes = {
    'transaction_date': str,
    'atm_address': str,
    'country': str,
    'city': str,
    'amount': np.float32,
    'currency': np.float32,
    'mcc': str,
    'customer_id': str,
    'pos_address': str,
    'atm_address': str,
    'pos_adress_lat': np.float32,
    'pos_adress_lon': np.float32,
    'pos_address_lat': np.float32,
    'pos_address_lon': np.float32,
    'atm_address_lat': np.float32,
    'atm_address_lon': np.float32,
    'home_add_lat': np.float32,
    'home_add_lon': np.float32,
    'work_add_lat': np.float32,
    'work_add_lon': np.float32,
}

# для экономии памяти будем загружать только часть атрибутов транзакций
usecols_train = ['customer_id','transaction_date','amount','country', 'city', 'currency', 'mcc', 'pos_adress_lat', 'pos_adress_lon', 'atm_address_lat', 'atm_address_lon','home_add_lat','home_add_lon','work_add_lat','work_add_lon']
usecols_test = ['customer_id','transaction_date','amount','country', 'city', 'currency', 'mcc', 'pos_address_lat', 'pos_address_lon', 'atm_address_lat', 'atm_address_lon']

## Читаем train_set, test_set, соединяем в один датасет

In [3]:
dtypes = {
    'transaction_date': str,
    'atm_address': str,
    'country': str,
    'city': str,
    'amount': np.float32,
    'currency': np.float32,
    'mcc': str,
    'customer_id': str,
    'pos_address': str,
    'atm_address': str,
    'pos_adress_lat': np.float32,
    'pos_adress_lon': np.float32,
    'pos_address_lat': np.float32,
    'pos_address_lon': np.float32,
    'atm_address_lat': np.float32,
    'atm_address_lon': np.float32,
    'home_add_lat': np.float32,
    'home_add_lon': np.float32,
    'work_add_lat': np.float32,
    'work_add_lon': np.float32,
}

rnm = {
    'atm_address_lat': 'atm_lat',
    'atm_address_lon': 'atm_lon',
    'pos_adress_lat': 'pos_lat',
    'pos_adress_lon': 'pos_lon',
    'pos_address_lat': 'pos_lat',
    'pos_address_lon': 'pos_lon',
    'home_add_lat': 'home_lat',
    'home_add_lon': 'home_lon',
    'work_add_lat': 'work_lat',
    'work_add_lon': 'work_lon',
}

In [4]:
df_train = pd.read_csv('../data/train_set.csv', dtype=dtypes)
df_test = pd.read_csv('../data/test_set.csv', dtype=dtypes)

df_train.rename(columns=rnm, inplace=True)
df_test.rename(columns=rnm, inplace=True)

In [ ]:
# удалим чувак с множественными адресами
# print(df_train.shape)
# gb = df_train.groupby('customer_id')['work_lat'].agg('nunique') 
# cid_incorrect = gb[gb == 2].index
# df_train = df_train[~df_train.customer_id.isin(cid_incorrect.values)]
# print(df_train.shape)
# gb = df_train.groupby('customer_id')['home_lat'].agg('nunique') 
# cid_incorrect = gb[gb == 2].index
# df_train = df_train[~df_train.customer_id.isin(cid_incorrect.values)]
# print(df_train.shape)

In [5]:
# соединяем test/train в одном DataFrame
df_train['is_train'] = np.int32(1)
df_test['is_train'] = np.int32(0)
df_all = pd.concat([df_train, df_test])

del df_train, df_test

### Обрабатываем дату транзакции и категориальные признаки

In [6]:
df_all['currency'] = df_all['currency'].fillna(-1).astype(np.int32)
df_all['mcc'] = df_all['mcc'].apply(lambda x: int(x.replace(',', ''))).astype(np.int32)
df_all['city'] = df_all['city'].factorize()[0].astype(np.int32)
df_all['country'] = df_all['country'].factorize()[0].astype(np.int32)

# удаляем транзакции без даты
df_all = df_all[~df_all['transaction_date'].isnull()]
df_all['transaction_date'] =  pd.to_datetime(df_all['transaction_date'], format='%Y-%m-%d')

### Фичи для даты

In [7]:
df_all['month'] = df_all.transaction_date.dt.month
df_all['day'] = df_all.transaction_date.dt.day
df_all['dayofyear'] = df_all.transaction_date.dt.dayofyear
df_all['dayofweek'] = df_all.transaction_date.dt.dayofweek

### Приводим адрес транзакции для pos и atm-транзакций к единообразному виду
Просто объединяем в одну колонку и добавляем фичу - это атм или пос

In [8]:
df_all['is_atm'] = (~df_all['atm_lat'].isnull()).astype(np.int8)
df_all['is_pos'] = (~df_all['pos_lat'].isnull()).astype(np.int8)

df_all['add_lat'] = df_all['atm_lat'].fillna(0) + df_all['pos_lat'].fillna(0)
df_all['add_lon'] = df_all['atm_lon'].fillna(0) + df_all['pos_lon'].fillna(0)

df_all.drop(['atm_lat','atm_lon','pos_lat','pos_lon'], axis=1, inplace=True)

df_all = df_all[~((df_all['add_lon'] == 0) & (df_all['add_lon'] == 0))]

In [9]:
%%time
# грязный хак, чтобы не учить КНН на новом юзере каждый раз
df_all['fake_customer_id'] = (pd.factorize(df_all.customer_id)[0] + 1) * 100

points = df_all[['fake_customer_id', 'add_lat', 'add_lon']].drop_duplicates().values
neigh = NearestNeighbors(2, radius=100000)

# расстояние до уникальных точек
# neigh.fit(np.unique(points, axis=1))
neigh.fit(points) 

distances, indices = neigh.kneighbors(df_all[['fake_customer_id', 'add_lat', 'add_lon']].values)
df_all['distance_to_nearest_point'] = distances[:, 1]
del df_all['fake_customer_id']

CPU times: user 4.82 s, sys: 107 ms, total: 4.92 s
Wall time: 4.92 s


In [11]:
# фичи с кластерами из тинькова
dfs = []
customers = df_all.customer_id.unique()
np_values = df_all[['customer_id', 'add_lat', 'add_lon']].values

for i in tqdm(range(len(customers))):
    customer = customers[i]
    points = np_values[np_values[:, 0] == customer][:, 1:]
    # оцениваем число кластеров
#     avgs = []
#     max_cluster = min(10,len(points))
#     for i in range(2,max_cluster):
#         kmeans = KMeans(n_clusters=i, random_state=2).fit(points)
#         labels = kmeans.labels_
#         silhouette_avg = silhouette_score(points, labels)
#         avgs.append(silhouette_avg)
        
#     if max_cluster == 2:
#         kmeans = KMeans(n_clusters=2, random_state=2).fit(points)
#         labels = kmeans.labels_
#         silhouette_avg = silhouette_score(points, labels)
#         avgs.append(silhouette_avg)
        
#     n_cluster = avgs.index(max(avgs)) + 2 # так как индексы с 0 а кластеры с 2
    # получаем лучший кластер
    if np.unique(points).size == 2:
        dfs.append(np.zeros((len(points), 4)))
        continue
    n_cluster = 2
    kmeans = KMeans(n_clusters=n_cluster, random_state=2).fit(points)
    #kmeans = AgglomerativeClustering(n_clusters=n_cluster,linkage='average').fit(points)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    silhouette_avg = silhouette_score(points, labels)
    # формируем датафрейм
    sample_silhouette_values = silhouette_samples(points, labels)
#     cluster_df = pd.DataFrame(data=np.vstack((labels, sample_silhouette_values)).T,columns=['label','score'])
#     cluster_df.label = cluster_df.label.astype(np.int32)
#     cluster_df['cluster_center_lat'] = cluster_df.apply(lambda row: centers[int(row['label'])][0], axis=1)
#     cluster_df['cluster_center_lon'] = cluster_df.apply(lambda row: centers[int(row['label'])][1], axis=1)
    arr_label_score = np.vstack((labels, sample_silhouette_values)).T
    arr_label_score = np.hstack([arr_label_score, centers[labels]])
    dfs.append(arr_label_score)

100%|██████████| 19997/19997 [14:11<00:00, 23.47it/s]


In [12]:
df_cluster = pd.DataFrame(np.vstack(dfs), columns=['cl_label','cl_score', 'cl_lat', 'cl_lon'])
df_all.reset_index(inplace=True, drop=True)
df_all = pd.concat([df_all, df_cluster], axis=1)

In [13]:
df_all.to_csv('../data/df_all_1.csv', index=None)

### Генерируем признаки is_home, is_work
TODO: удалить чуваков у которых несколько домов

In [15]:
lat = df_all['home_lat'] - df_all['add_lat']
lon = df_all['home_lon'] - df_all['add_lon']

df_all['is_home'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int8)
df_all['has_home'] = (~df_all['home_lon'].isnull()).astype(np.int8)

lat = df_all['work_lat'] - df_all['add_lat']
lon = df_all['work_lon'] - df_all['add_lon']
df_all['is_work'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int8)
df_all['has_work'] = (~df_all['work_lon'].isnull()).astype(np.int8)

df_all.drop(['work_lat','work_lon','home_lat','home_lon'], axis=1, inplace=True)

### Генерируем категориальный признак для адреса

In [16]:
df_all['address'] = df_all['add_lat'].apply(lambda x: "%.02f" % x) + ';' + df_all['add_lon'].apply(lambda x: "%.02f" % x)
df_all['address'] = df_all['address'].factorize()[0].astype(np.int32)

### Генерируем несколько абонентских фич

In [17]:
# количество транзакций каждого клиента
df_all = df_all.merge(df_all.groupby('customer_id')['amount'].count().reset_index(name='cid_trans_count'), how='left')
df_all['cid_trans_count'] = df_all['cid_trans_count'].astype(np.int32)

df_all = df_all.merge(df_all.groupby(['customer_id','address'])['amount'].count().reset_index(name='cid_add_trans_count'), 
                      how='left')
df_all['cid_add_trans_count'] = df_all['cid_add_trans_count'].astype(np.int32)

# какая часть транзакций клиента приходится на данный адрес
# TODO: БОЛЬШЕ ТАКИХ ФИЧ
df_all['ratio1'] = df_all['cid_add_trans_count'] / df_all['cid_trans_count']

## Мои фичи

In [18]:
# добавим признаки после групбая
df_gb = df_all[['customer_id','amount', 'add_lat', 'add_lon']].groupby('customer_id')
coord_stat_df = df_gb.agg(['mean', 'max', 'min'])
coord_stat_df['transactions_per_user'] = df_gb.agg('size')
coord_stat_df.columns = ['_'.join(col).strip() for col in coord_stat_df.columns.values]
coord_stat_df.reset_index(inplace=True)
df_all = pd.merge(df_all, coord_stat_df, on='customer_id', how='left')

In [19]:
cols = ['add_lat', 'add_lon']
types = ['min', 'max', 'mean']
for c in cols:
    for t in types:
        df_all['{}_diff_{}'.format(c, t)] = np.abs(df_all[c] - df_all['{}_{}'.format(c, t)])

In [20]:
# разности 
df_all['lat_diff_cluster_lat'] = np.abs(df_all['add_lat'] - df_all['cl_lat'])
df_all['lon_diff_cluster_lon'] = np.abs(df_all['add_lon'] - df_all['cl_lon'])

In [21]:
df_all = pd.concat([df_all, pd.get_dummies(df_all['mcc'], prefix='mcc')], axis=1)
del df_all['mcc']

# LightGBM

In [22]:
df_all = df_all.loc[:,~df_all.columns.duplicated()]

In [23]:
from sklearn.model_selection import train_test_split

ys = ['is_home', 'is_work']
drop_cols = ['atm_address', 'customer_id', 'pos_address', 'terminal_id', 'transaction_date', 
             'is_home' ,'has_home', 'is_work', 'has_work', 'is_train']

drop_cols += ['pred:is_home', 'pred:is_work']
y_cols = ['is_home', 'is_work']
usecols = df_all.drop(drop_cols, 1, errors='ignore').columns

In [24]:
params = {
    'objective': 'binary',
    'num_leaves': 63,
    'learning_rate': 0.01,
    'metric' : 'binary_logloss',
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'num_threads': 12,
    'verbose': 0,
}

model = {}

In [25]:
y_col = 'is_home'

cust_train = df_all[df_all['is_train']==1].groupby('customer_id')[y_col.replace('is_','has_')].max()
cust_train = cust_train[cust_train > 0].index

cust_train, cust_valid = train_test_split(cust_train, test_size=0.2, shuffle=True, random_state=111)

df_train = pd.DataFrame(cust_train, columns=['customer_id']).merge(df_all, how='left')
df_valid = pd.DataFrame(cust_valid, columns=['customer_id']).merge(df_all, how='left')

lgb_train = lgb.Dataset(df_train[usecols], df_train[y_col])
lgb_valid = lgb.Dataset(df_valid[usecols], df_valid[y_col])

gbm_h = lgb.train(params,
                lgb_train,
                valid_sets=[lgb_valid],
                num_boost_round=2000,
                verbose_eval=30,
                early_stopping_rounds=300)

model[y_col] = gbm_h

Training until validation scores don't improve for 300 rounds.
[30]	valid_0's binary_logloss: 0.598695
[60]	valid_0's binary_logloss: 0.541358
[90]	valid_0's binary_logloss: 0.504476
[120]	valid_0's binary_logloss: 0.479779
[150]	valid_0's binary_logloss: 0.46284
[180]	valid_0's binary_logloss: 0.450269
[210]	valid_0's binary_logloss: 0.441381
[240]	valid_0's binary_logloss: 0.434503
[270]	valid_0's binary_logloss: 0.430182
[300]	valid_0's binary_logloss: 0.427002
[330]	valid_0's binary_logloss: 0.424357
[360]	valid_0's binary_logloss: 0.422388
[390]	valid_0's binary_logloss: 0.420946
[420]	valid_0's binary_logloss: 0.420155
[450]	valid_0's binary_logloss: 0.419309
[480]	valid_0's binary_logloss: 0.418406
[510]	valid_0's binary_logloss: 0.417984
[540]	valid_0's binary_logloss: 0.417715
[570]	valid_0's binary_logloss: 0.41724
[600]	valid_0's binary_logloss: 0.416979
[630]	valid_0's binary_logloss: 0.416843
[660]	valid_0's binary_logloss: 0.416772
[690]	valid_0's binary_logloss: 0.416764

In [26]:
y_col = 'is_work'

cust_train = df_all[df_all['is_train']==1].groupby('customer_id')[y_col.replace('is_','has_')].max()
cust_train = cust_train[cust_train > 0].index

cust_train, cust_valid = train_test_split(cust_train, test_size=0.2, shuffle=True, random_state=111)



df_train = pd.DataFrame(cust_train, columns=['customer_id']).merge(df_all, how='left')
df_valid = pd.DataFrame(cust_valid, columns=['customer_id']).merge(df_all, how='left')

lgb_train = lgb.Dataset(df_train[usecols], df_train[y_col])
lgb_valid = lgb.Dataset(df_valid[usecols], df_valid[y_col])

gbm_w = lgb.train(params,
                lgb_train,
                valid_sets=[lgb_valid],
                num_boost_round=2000,
                verbose_eval=30,
                early_stopping_rounds=300)

model[y_col] = gbm_w

Training until validation scores don't improve for 300 rounds.
[30]	valid_0's binary_logloss: 0.577558
[60]	valid_0's binary_logloss: 0.506715
[90]	valid_0's binary_logloss: 0.460817
[120]	valid_0's binary_logloss: 0.42959
[150]	valid_0's binary_logloss: 0.408391
[180]	valid_0's binary_logloss: 0.393525
[210]	valid_0's binary_logloss: 0.382874
[240]	valid_0's binary_logloss: 0.375798
[270]	valid_0's binary_logloss: 0.370509
[300]	valid_0's binary_logloss: 0.366902
[330]	valid_0's binary_logloss: 0.363107
[360]	valid_0's binary_logloss: 0.360295
[390]	valid_0's binary_logloss: 0.357828
[420]	valid_0's binary_logloss: 0.355896
[450]	valid_0's binary_logloss: 0.354187
[480]	valid_0's binary_logloss: 0.353145
[510]	valid_0's binary_logloss: 0.352088
[540]	valid_0's binary_logloss: 0.35114
[570]	valid_0's binary_logloss: 0.350587
[600]	valid_0's binary_logloss: 0.350096
[630]	valid_0's binary_logloss: 0.34949
[660]	valid_0's binary_logloss: 0.34942
[690]	valid_0's binary_logloss: 0.349329
[

In [ ]:
lgb.plot_importance(gbm_w, max_num_features=15)

In [27]:
def _best(x):
    ret = None
    for col in ys:
        pred = ('pred:%s' % col)
        if pred in x:
            i = (x[pred].idxmax())
            cols = [pred, 'add_lat', 'add_lon']
            if col in x:
                cols.append(col)
            tmp = x.loc[i,cols]
            tmp.rename({
                'add_lat':'%s:add_lat' % col,
                'add_lon':'%s:add_lon' % col,
            }, inplace = True)
            if ret is None:
                ret = tmp
            else:
                ret = pd.concat([ret, tmp])
    return ret

def predict_proba(dt, ys=['is_home', 'is_work']):
    for col in ys:
        pred = ('pred:%s' % col)
        dt[pred] = model[col].predict(dt[usecols])
    return dt.groupby('customer_id').apply(_best).reset_index()

def score(dt, ys=['is_home', 'is_work']):
    dt_ret = predict_proba(dt, ys)
    mean = 0.0
    for col in ys:
        col_mean = dt_ret[col].mean()
        mean += col_mean
    if len(ys) == 2:
        mean = mean / len(ys)
    return mean

In [28]:
print ("Train accuracy:", score(df_train, ys=['is_home']))
print ("Test accuracy:", score(df_valid, ys=['is_home']))

print ("Train accuracy:", score(df_train, ys=['is_work']))
print ("Test accuracy:", score(df_valid, ys=['is_work']))

Train accuracy: 0.5087251575375666
Test accuracy: 0.5096899224806202
Train accuracy: 0.37930198739699467
Test accuracy: 0.3430232558139535


# Predict

In [ ]:
cust_test = df_all[df_all['is_train'] == 0]['customer_id'].unique()
df_test = pd.DataFrame(cust_test, columns = ['customer_id']).merge(df_all, how = 'left')
df_test = predict_proba(df_test)
df_test.rename(columns = {
        'customer_id':'_ID_',
        'is_home:add_lat': '_HOME_LAT_',
        'is_home:add_lon': '_HOME_LON_',
        'is_work:add_lat': '_WORK_LAT_',
        'is_work:add_lon': '_WORK_LON_'}, inplace = True)
df_test = df_test[['_ID_', '_WORK_LAT_', '_WORK_LON_', '_HOME_LAT_', '_HOME_LON_']]

df_test.head()

# Формируем submission-файл

In [ ]:
# Заполняем пропуски
df_ = pd.read_csv('../data/test_set.csv', dtype=dtypes, usecols=['customer_id'])
submission = pd.DataFrame(df_['customer_id'].unique(), columns=['_ID_'])

submission = submission.merge(df_test, how='left').fillna(0)
# Пишем файл submission
submission.to_csv('../submissions/base_3_50_34.csv', index=None)